### QUESTÃO A 

In [0]:
%sql
-- QUESTÃO A
SELECT 
  Provider,
  COUNT (CASE WHEN Status = "PROVIDER_FAILED" THEN 1 END) AS failure_counts,
  ROUND ( 
    (COUNT (CASE WHEN Status = "PROVIDER_FAILED" THEN 1 END)/ SUM(COUNT (CASE WHEN Status = "PROVIDER_FAILED" THEN 1 END)) OVER ()) *100,
  2) AS percent_total_failures, -- do total de failures, o quanto o provider especifico representou
  COUNT (Provider) AS provider_executions,
  ROUND ( 
    (COUNT (CASE WHEN Status = "PROVIDER_FAILED" THEN 1 END)/ COUNT (Provider)) *100,
  2) AS percent_provider_error --de todas execuções do provider, qual percentual de falha ele apresentou
FROM biometry_execution
WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros da tabela
GROUP BY Provider
ORDER BY failure_counts DESC 



Provider,failure_counts,percent_total_failures,provider_executions,percent_provider_error
A,40,78.43,496,8.06
B,8,15.69,40,20.0
C,3,5.88,9,33.33


### QUESTÃO B

In [0]:
%sql
--QUESTÃO B
--gerando as execuções biométricas válidas, isto é, sem erro no input "Similarity"
with valid_biometry_executions AS (
  SELECT 
    DISTINCT Session_ID 
  from biometry_execution
  WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros 
)

--fazendo contagens e calculando os devidos valores totais para cada categoria, bem como quantidade de status not match e o percentual
SELECT
  Category, 
  COUNT(CASE WHEN b.Status = "NOT_MATCH" THEN 1 END) as not_match_count,
  ROUND ( 
    (COUNT(CASE WHEN b.Status = "NOT_MATCH" THEN 1 END)/ SUM(COUNT(CASE WHEN b.Status = "NOT_MATCH" THEN 1 END)) OVER ()) *100,
  2) AS percent_not_match, --de todos os not matches, o quanto a referida categoria representou
  COUNT (Category) AS drivers_in_category,
  ROUND ( 
    (COUNT(CASE WHEN b.Status = "NOT_MATCH" THEN 1 END)/ COUNT(Category)) *100,
  2) AS percent_category_not_match --dos drivers com biometria nessa categoria, qual foi o percentual de not match
  
FROM biometry b
LEFT JOIN  drivers d
  ON b.Driver_ID = d.Driver_ID 
INNER JOIN valid_biometry_executions be 
  ON b.Session_ID = be.Session_ID -- trazendo somente valid executions, eliminando o dado incorreto
WHERE b.Status <> "PROVIDER_FAILED" -- asumindo que os casos finais com provider failed em todos os providers, foi erro de sistema e portanto não deve ser contabilizado no calculo de percentual de not match dentro do universo de todos os testes bem sucedidos
GROUP BY Category 
ORDER BY not_match_count DESC 

Category,not_match_count,percent_not_match,drivers_in_category,percent_category_not_match
Bronze,25,40.98,149,16.78
Prata,20,32.79,148,13.51
Iniciante,11,18.03,94,11.7
Ouro,4,6.56,79,5.06
Diamante,1,1.64,23,4.35


### QUESTÃO C

In [0]:
%sql
--QUESTÃO C - Simple and straightforward solution
WITH similarity_increase AS (
SELECT 
  be.Session_ID,
  Provider,
  be.Status,
  TRY_CAST (REPLACE ( Similarity, '0.', '') AS FLOAT)/100 AS similarity_cleaned, --tratamentos na coluna de similarity, para ler corretamente os números e tratar textos como null
  CASE 
    WHEN similarity_cleaned >= 0.9 THEN "MATCH"
    WHEN similarity_cleaned < 0.9 THEN "NOT_MATCH"
    ELSE "PROVIDER_FAILED"
  END AS new_status_90,
  ROW_NUMBER() OVER (PARTITION BY be.Session_ID ORDER BY Provider DESC) AS rn
FROM biometry_execution be
--LEFT JOIN biometry b
  --ON be.Session_ID = b.Session_ID
WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros 
AND be.Status <> "PROVIDER_FAILED"
)


SELECT 
  COUNT(Status) AS total_biometry_executions, 
  COUNT(CASE WHEN Status = 'MATCH' THEN 1 END) AS old_match_count, 
  ROUND(COUNT(CASE WHEN Status = "MATCH" THEN 1 END)/ COUNT(DISTINCT Session_ID),2) AS old_match_percent,
  COUNT(CASE WHEN new_status_90 = 'MATCH' THEN 1 END) AS new_match_count, 
  ROUND(COUNT(CASE WHEN new_status_90 = "MATCH" THEN 1 END)/ COUNT(DISTINCT Session_ID),2) AS new_match_percent
  
FROM similarity_increase
WHERE rn = 1 --como row number foi contado a partir da ordenação decrescente de Provider, garante que vai pegar o resultado da última execução para aquela Session_ID


total_biometry_executions,old_match_count,old_match_percent,new_match_count,new_match_percent
493,432,0.88,333,0.68


In [0]:
%sql
--QUESTÃO C.1 - extra information on each provider on new match rates
WITH similarity_increase AS (
SELECT 
    be.Session_ID,
    Provider AS Provider_New,
    be.Status,
    TRY_CAST (REPLACE ( Similarity, '0.', '') AS FLOAT)/100 AS similarity_cleaned, --tratamentos na coluna de similarity, para ler corretamente os números e tratar textos como null
    CASE 
      WHEN similarity_cleaned >= 0.9 THEN "MATCH"
      WHEN similarity_cleaned < 0.9 THEN "NOT_MATCH"
      ELSE "PROVIDER_FAILED"
    END AS new_status_90,
    ROW_NUMBER() OVER (PARTITION BY be.Session_ID ORDER BY Provider DESC) AS rn
FROM biometry_execution be
--LEFT JOIN biometry b
  --ON be.Session_ID = b.Session_ID
WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros 
AND be.Status <> "PROVIDER_FAILED" --pois queremos contabilizar apenas execuções finais válidas para construção da nova biometry 
)

SELECT 
  provider_new,
  COUNT (new_status_90),
  COUNT(CASE WHEN new_status_90 = 'MATCH' THEN 1 END) AS new_match_count, 
  ROUND(COUNT(CASE WHEN new_status_90 = "MATCH" THEN 1 END)/ COUNT(new_status_90),2) AS provider_new_match_percent,
  ROUND(COUNT(CASE WHEN new_status_90 = "MATCH" THEN 1 END) / SUM(COUNT(CASE WHEN new_status_90 = "MATCH" THEN 1 END)) OVER (),2) AS match_share,
  ROUND(COUNT(CASE WHEN new_status_90 = "MATCH" THEN 1 END) / SUM(COUNT(new_status_90)) OVER (),2) AS total_match_percent
FROM similarity_increase
WHERE rn = 1 --como row number foi contado a partir da ordenação decrescente de Provider, garante que vai pegar o resultado da última execução para aquela Session_ID
GROUP BY provider_new
ORDER BY provider_new ASC 



provider_new,count(new_status_90),new_match_count,provider_new_match_percent,match_share,total_match_percent
A,456,311,0.68,0.93,0.63
B,31,18,0.58,0.05,0.04
C,6,4,0.67,0.01,0.01


In [0]:
%sql
-- QUESTÃO C.2 - extra information on each provider on old match rates, to compare with new match rates
with 
--gerando as execuções biométricas válidas, isto é, sem erro no output "Similarity"
valid_biometry_executions AS (
  SELECT 
    DISTINCT Session_ID
  from biometry_execution
  WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros 
),

--consolidando todos os dados necesssários com devidos filtros e sanitização de bases utilizadas
base as (
SELECT
  b.*,
  be.Provider
FROM biometry b 
LEFT JOIN biometry_execution be  -- trazendo com esse join o provider que resultou no status final daquela session
  ON b.Session_ID = be.Session_ID 
  AND b.Status = be.Status
INNER JOIN valid_biometry_executions v -- para remover o caso de erro onde Similarity = "NOT MATCH"
  ON b.Session_ID = v.Session_ID
WHERE b.Status <> "PROVIDER_FAILED" -- asumindo que os casos finais com provider failed em todos os providers, foi erro de sistema e portanto não deve ser contabilizado no calculo de percentual de not match dentro do universo de todos os testes bem sucedidos

)
SELECT 
  Provider,
  COUNT (Status),
  COUNT(CASE WHEN Status = "MATCH" THEN 1 END) as match_count,
  ROUND(COUNT(CASE WHEN Status = "MATCH" THEN 1 END)/ COUNT(Status),2) AS provider_match_percent,
  ROUND(COUNT(CASE WHEN Status = "MATCH" THEN 1 END) / SUM(COUNT(CASE WHEN Status = "MATCH" THEN 1 END)) OVER (),2) AS match_share,
  ROUND(COUNT(CASE WHEN Status = "MATCH" THEN 1 END) / SUM(COUNT(Status)) OVER (),2) AS total_match_percent
FROM base 
GROUP BY Provider 
ORDER BY Provider ASC 

Provider,count(Status),match_count,provider_match_percent,match_share,total_match_percent
A,456,398,0.87,0.92,0.81
B,31,28,0.9,0.06,0.06
C,6,6,1.0,0.01,0.01


### QUESTÃO D

In [0]:
%sql
-- QUESTÃO D.1 - Análise do impacto da biometria no status dos pedidos RESPOSTA DA PERGUNTA - percents

--filtrando erros da base de pedidos; selecionando e transformando colunas a serem utilizadas
WITH orders AS (
  SELECT 
    Driver_ID AS driver_id,
    DATE(Order_DT) AS data_pedido,
    Order_Status AS order_status,
    Order_ID AS order_id
  FROM orders
  WHERE order_status <> "#N/D" -- Remover casos com erro pois não sabemos qual foi o status final do pedido
),

--gerando as execuções biométricas válidas, isto é, sem erro no input "Similarity"
valid_biometry_executions AS (
  SELECT DISTINCT Session_ID 
  FROM biometry_execution
   WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros 
),

--gerando a tabela com infos de biometria, filtrando para resultados finais válidos de biometria 
biometries AS (
  SELECT 
    Driver_ID AS driver_id,
    DATE(b.Event_Dt) AS data_biometria,
    Status AS biometry_status
  FROM biometry b 
  INNER JOIN valid_biometry_executions be
    ON b.Session_ID = be.Session_ID 
  WHERE b.Status <> "PROVIDER_FAILED" -- asumindo que os casos finais com provider failed em todos os providers, foi erro de sistema e portanto não deve ser contabilizado como biometria realizada 
),

--consolidando todos os dados necesssários após consumir das bases sanitizadas, e realizando primeira contagem de pedidos
base AS (
  SELECT 
    o.driver_id,
    b.biometry_status,
    order_status,
    COUNT(DISTINCT o.order_id) AS order_num

  FROM orders o
  LEFT JOIN biometries b
    ON o.driver_id = b.driver_id
  WHERE biometry_status IS NOT NULL --dessa forma, somente observando pedidos os quais o entregador tem resultado de biometria na base tratada de biometry, podendo comparar corretamente entre match e not match
  GROUP BY 1,2,3
),

--agregando por status pedido e status biometria
calculations AS (
SELECT 
  order_status,
  biometry_status,
  SUM(order_num) AS total_orders

FROM base
GROUP BY 1,2
ORDER BY order_status, biometry_status
),

--gerando coluna de total por status de biometria - total de pedidos para driver com status match e total de pedidos para status not match
last_calculations AS (
SELECT 
    order_status,
    biometry_status,
    total_orders,
    SUM(total_orders) OVER (PARTITION BY biometry_status) AS total_pedidos_per_biometry_status
  FROM calculations
  GROUP BY 1,2,3

),

driver_count AS (
  SELECT 
    COUNT(DISTINCT driver_id) AS driver_count_per_status,
    biometry_status
  FROM base 
  GROUP BY biometry_status 
)

SELECT 
  lc.*,
  driver_count_per_status,
  ROUND((total_orders/total_pedidos_per_biometry_status )*100,2) AS percent_per_biometry,
  ROUND((total_orders/driver_count_per_status ),2) AS orders_per_driver

FROM last_calculations lc
LEFT JOIN driver_count dc
  ON lc.biometry_status = dc.biometry_status
ORDER BY 2

order_status,biometry_status,total_orders,total_pedidos_per_biometry_status,driver_count_per_status,percent_per_biometry,orders_per_driver
CANCELLED,MATCH,885,5897,432,15.01,2.05
CONCLUDED,MATCH,5012,5897,432,84.99,11.6
CANCELLED,NOT_MATCH,137,802,61,17.08,2.25
CONCLUDED,NOT_MATCH,665,802,61,82.92,10.9


In [0]:
%sql
-- QUESTÃO D.2 - Análise do impacto da biometria no status dos pedidos RESPOSTA DA PERGUNTA - PEARSON CORREL

--filtrando erros da base de pedidos; selecionando e transformando colunas a serem utilizadas
WITH orders AS (
  SELECT 
    Driver_ID AS driver_id,
    DATE(Order_DT) AS data_pedido,
    Order_Status AS order_status,
    Order_ID AS order_id
  FROM orders
  WHERE order_status <> "#N/D" -- Remover casos com erro pois não sabemos qual foi o status final do pedido
),

--gerando as execuções biométricas válidas, isto é, sem erro no output "Similarity"
valid_biometry_executions AS (
  SELECT DISTINCT Session_ID 
  FROM biometry_execution
   WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros 
),

--gerando a tabela com infos de biometria, filtrando para resultados finais válidos de biometria 
biometries AS (
  SELECT 
    Driver_ID AS driver_id,
    DATE(b.Event_Dt) AS data_biometria,
    Status AS biometry_status
  FROM biometry b 
  INNER JOIN valid_biometry_executions be
    ON b.Session_ID = be.Session_ID 
  WHERE b.Status <> "PROVIDER_FAILED" -- asumindo que os casos finais com provider failed em todos os providers, foi erro de sistema e portanto não deve ser contabilizado como biometria válida realizada 
),

--consolidando todos os dados necesssários após consumir das bases sanitizadas, e realizando cálculos e gerando colunas dummy
base AS (
  SELECT 
    o.driver_id,
    b.biometry_status,
    
    -- Número de pedidos cancelados por entregador
    COUNT(CASE WHEN o.order_status = 'CANCELLED' THEN 1 END) AS cancelled_orders,

    COUNT(CASE WHEN o.order_status = 'CONCLUDED' THEN 1 END) AS completed_orders,
    
    -- Bool indicando biometria com falha do entregador
    MAX(CASE WHEN b.biometry_status = 'NOT_MATCH' THEN 1 ELSE 0 END) AS failed_biometry,

    -- Bool indicando biometria com sucesso do entregador
    MAX(CASE WHEN b.biometry_status = 'MATCH' THEN 1 ELSE 0 END) AS completed_biometry,

    --Bool indicando falta de biometria na base biometry para aquele entregador
    MAX(CASE WHEN b.driver_id IS NULL THEN 1 ELSE 0 END ) AS no_recent_biometry


  FROM orders o
  LEFT JOIN biometries b
    ON o.driver_id = b.driver_id
  WHERE biometry_status IS NOT NULL --dessa forma, somente observando pedidos os quais o entregador tem resultado de biometria na base tratada de biometry, podendo comparar corretamente entre match e not match
  GROUP BY o.driver_id, b.biometry_status
)

SELECT 
  ROUND (corr(cancelled_orders, failed_biometry),4) AS fail_cancel_corr,
  ROUND (corr(cancelled_orders, completed_biometry),4) AS match_cancel_corr
FROM base
WHERE no_recent_biometry = 0

fail_cancel_corr,match_cancel_corr
0.0416,-0.0416


In [0]:
%sql
--QUESTÃO D - b) padrão nas datas
WITH 
--gerando as execuções biométricas válidas, isto é, sem erro no output "Similarity"
valid_biometry_executions AS ( 
  SELECT DISTINCT Session_ID 
  FROM biometry_execution
   WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros 
),
--gerando a tabela com infos de biometria, filtrando para resultados finais válidos de biometria 
biometries AS ( 
  SELECT 
    Driver_ID AS driver_id,
    DATE(b.Event_Dt) AS data_biometria,
    Status AS biometry_status
  FROM biometry b 
  INNER JOIN valid_biometry_executions be --trazendo então somente as biometrias finais que tiveram pelo menos uma execução válida de biometria
    ON b.Session_ID = be.Session_ID 
  WHERE b.Status <> "PROVIDER_FAILED" -- asumindo que os casos finais com provider failed em todos os providers, foi erro de sistema e portanto não deve ser contabilizado como biometria válida realizada 
),

--consolidando todos os dados necesssários após consumir das bases sanitizadas, contando as ocorrências de match e not match para diversas granularidades de data
date_agregations AS (
SELECT 
  month(data_biometria) AS month_biometria,
  day(data_biometria) AS day_biometria,
  dayofweek(data_biometria) AS num_week_day_biometria,
  date_format(data_biometria, 'EEEE') AS week_day_biometria,
  COUNT(CASE WHEN biometry_status = "MATCH" THEN 1 END) AS matches,
  COUNT(CASE WHEN biometry_status = "NOT_MATCH" THEN 1 END) AS not_matches

FROM biometries
GROUP BY 1,2,3,4
ORDER BY 2,3 ASC 
)

-- após testar os diversas granularidades de data, se chega na conclusão - pela melhor distribuição de data points - e na melhor leitura de padrões, que observar os dados pelo week day nos dá o melhor resultado, conforme analisado a seguir:
SELECT 
  num_week_day_biometria,
  week_day_biometria,
  ROUND((SUM(not_matches)/ (SUM(matches) + SUM(not_matches)))*100,2) AS not_match_percent,
  SUM(not_matches), 
  SUM(matches)
FROM date_agregations
GROUP BY 1,2
ORDER BY 1 ASC 


num_week_day_biometria,week_day_biometria,not_match_percent,sum(not_matches),sum(matches)
1,Sunday,13.92,11,68
2,Monday,19.48,15,62
3,Tuesday,15.0,12,68
4,Wednesday,13.7,10,63
5,Thursday,6.45,4,58
6,Friday,5.71,4,66
7,Saturday,9.62,5,47


In [0]:
%sql
-- QUESTÃO D- b)- Análise extra de pedidos por dia da semana

--filtrando erros da base de pedidos; selecionando e transformando colunas a serem utilizadas
WITH orders AS (
  SELECT 
    Driver_ID AS driver_id,
    DATE(Order_DT) AS data_pedido,
    Order_Status AS order_status,
    Order_ID AS order_id
  FROM orders
  WHERE order_status <> "#N/D" -- Remover casos com erro pois não sabemos qual foi o status final do pedido
),

--gerando as execuções biométricas válidas, isto é, sem erro no output "Similarity"
valid_biometry_executions AS (
  SELECT DISTINCT Session_ID 
  FROM biometry_execution
   WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros 
),

--gerando a tabela com infos de biometria, filtrando para resultados finais válidos de biometria 
biometries AS (
  SELECT 
    Driver_ID AS driver_id,
    DATE(b.Event_Dt) AS data_biometria,
    Status AS biometry_status
  FROM biometry b 
  INNER JOIN valid_biometry_executions be
    ON b.Session_ID = be.Session_ID 
  WHERE b.Status <> "PROVIDER_FAILED" -- asumindo que os casos finais com provider failed em todos os providers, foi erro de sistema e portanto não deve ser contabilizado como biometria válida realizada 
),

--consolidando todos os dados necesssários após consumir das bases sanitizadas, e realizando cálculos e gerando colunas dummy
base AS (
  SELECT 
    o.*
  FROM orders o
  LEFT JOIN biometries b
    ON o.driver_id = b.driver_id
  WHERE biometry_status IS NOT NULL --dessa forma, somente observando pedidos os quais o entregador tem resultado de biometria na base tratada de biometry, podendo comparar corretamente entre match e not match
),

date_agregations AS (
SELECT 
  month(data_pedido) AS mes_pedido,
  day(data_pedido) AS dia_pedido,
  dayofweek(data_pedido) AS num_week_day_pedido,
  date_format(data_pedido, 'EEEE') AS week_day_pedido,
  COUNT(CASE WHEN order_status = "CONCLUDED" THEN 1 END) AS pedidos_completos,
  COUNT(CASE WHEN order_status <> "CONCLUDED"  THEN 1 END) AS pedidos_cancelados

FROM base
GROUP BY 1,2,3,4
ORDER BY 2,3 ASC 
)

-- após testar os diversas granularidades de data, se chega na conclusão - pela melhor distribuição de data points - e na melhor leitura de padrões, que observar os dados pelo week day nos dá o melhor resultado, conforme analisado a seguir:
SELECT 
  num_week_day_pedido,
  week_day_pedido,
  ROUND((SUM(pedidos_cancelados)/ (SUM(pedidos_completos) + SUM(pedidos_cancelados)))*100,2) AS cancel_rate,
  SUM(pedidos_completos), 
  SUM(pedidos_cancelados),
  SUM(pedidos_completos) + SUM(pedidos_cancelados) AS pedidos_totais
FROM date_agregations
GROUP BY 1,2
ORDER BY 1 ASC 


num_week_day_pedido,week_day_pedido,cancel_rate,sum(pedidos_completos),sum(pedidos_cancelados),pedidos_totais
1,Sunday,13.05,746,112,858
2,Monday,15.02,747,132,879
3,Tuesday,15.21,1187,213,1400
4,Wednesday,14.19,756,125,881
5,Thursday,15.02,764,135,899
6,Friday,17.08,704,145,849
7,Saturday,17.15,773,160,933


In [0]:
%sql
-- QUESTÃO D - Extra info - Pearson corr para Category

WITH  
--filtrando erros da base de pedidos; selecionando e transformando colunas a serem utilizadas
orders_agg AS (
  SELECT 
    Driver_ID AS driver_id, 
    COUNT(CASE WHEN Order_Status = "CANCELLED" THEN 1 END) AS cancelled_orders,
    COUNT(CASE WHEN Order_Status = "CONCLUDED" THEN 1 END) AS completed_orders,
    COUNT(Order_ID) AS total_orders
  FROM orders
  WHERE order_status <> "#N/D" -- Remover casos com erro pois não sabemos qual foi o status final do pedido
  GROUP BY Driver_ID
  
),
--gerando as execuções biométricas válidas, isto é, sem erro no output "Similarity"
valid_biometry_executions AS (
  SELECT DISTINCT Session_ID 
  FROM biometry_execution
   WHERE TRY_CAST (coalesce(REPLACE( Similarity, '0.', ''), '0')  AS FLOAT) is not null-- Considerado um erro na tabela, pois diverge do padrão da coluna (numero que representa a similaridade) e pode poluir a análise, portanto: 1.remove o 0. antes dos numeros de similaridade, 2.substitui todos nulos por zero; 3.converte em float os zeros e números de similaridade; 4.valores não numéricos (contendo letras) agora serão null; 5.filtrar esses nulls para remover os erros 
),
--gerando a tabela com infos de biometria, filtrando para resultados finais válidos de biometria
biometries AS (
  SELECT 
    Driver_ID AS driver_id,
    DATE(b.Event_Dt) AS data_biometria,
    Status AS biometry_status
  FROM biometry b 
  INNER JOIN valid_biometry_executions be
    ON b.Session_ID = be.Session_ID 
  WHERE b.Status <> "PROVIDER_FAILED" -- asumindo que os casos finais com provider failed em todos os providers, foi erro de sistema e portanto não deve ser contabilizado como biometria válida realizada 
),

--informações dos motoristas a respeito qual categoria se encontra, por meio de dummies (boolean)
drivers_info AS (
  SELECT
  Driver_ID AS driver_id, 
  DATE (Register_Dt) AS register_date,
  CASE WHEN Category = "Prata" Then 1 ELSE 0 END as prata_bool,
  CASE WHEN Category = "Iniciante" Then 1 ELSE 0 END as iniciante_bool,
  CASE WHEN Category = "Diamante" Then 1 ELSE 0 END as diamante_bool,
  CASE WHEN Category = "Bronze" Then 1 ELSE 0 END as bronze_bool,
  CASE WHEN Category = "Ouro" Then 1 ELSE 0 END as ouro_bool
  FROM drivers
),

--consolidando todos os dados necesssários após consumir das bases sanitizadas
final_data AS (
  SELECT 
    d.*, 
    o.cancelled_orders,
    o.completed_orders,
    o.total_orders

  FROM drivers_info d
  LEFT JOIN orders_agg o
    ON d.driver_id = o.driver_id
  LEFT JOIN biometries b
    ON o.driver_id = b.driver_id
  WHERE biometry_status IS NOT NULL --dessa forma, somente observando pedidos os quais o entregador tem resultado de biometria na base tratada de biometry, podendo comparar corretamente entre match e not match
)

-- Cálculo da correlação entre quantidade de pedidos cancelados e a dummy que indica qual categoria o driver se encontra
SELECT
  ROUND(corr(bronze_bool, cancelled_orders), 4) AS corr_bronze_cancelled,
  ROUND(corr(prata_bool, cancelled_orders), 4) AS corr_prata_cancelled,
  ROUND(corr(iniciante_bool, cancelled_orders), 4) AS corr_iniciante_cancelled,
  ROUND(corr(ouro_bool, cancelled_orders), 4) AS corr_ouro_cancelled,
  ROUND(corr(diamante_bool, cancelled_orders), 4) AS corr_diamante_cancelled
FROM final_data

corr_bronze_cancelled,corr_prata_cancelled,corr_iniciante_cancelled,corr_ouro_cancelled,corr_diamante_cancelled
0.2689,-0.008,-0.3367,0.1035,-0.1212
